In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier


In [2]:
train = pd.read_csv('train_pre2.csv').fillna(' ')
test = pd.read_csv('test_pre2.csv').fillna(' ')

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"explanation,why,the,edits,made,under,my,userna...",0,0,0,0,0,0
1,000103f0d9cfb60f,"d'aww,he,matches,this,background,colour,i,am,s...",0,0,0,0,0,0
2,000113f07ec002fd,"hey,man,i,am,really,not,trying,to,edit,war,it,...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"more,i,can,not,make,any,real,suggestions,on,im...",0,0,0,0,0,0
4,0001d958c54c6e35,"you,sir,are,my,hero,any,chance,you,remember,wh...",0,0,0,0,0,0


In [4]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [5]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [6]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [7]:
print(len(word_vectorizer.vocabulary_),train_word_features.shape)

10000 (159571, 10000)


In [8]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=20000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [9]:
len(char_vectorizer.vocabulary_)

20000

In [10]:
print(train_word_features.shape,test_word_features.shape)

(159571, 10000) (153164, 10000)


In [11]:
print(train_char_features.shape,test_char_features.shape)

(159571, 20000) (153164, 20000)


In [12]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [13]:
test_features.shape

(153164, 30000)

In [55]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv = 3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9777873013877553
CV score for class severe_toxic is 0.9885500777123882
CV score for class obscene is 0.9899669940902244
CV score for class threat is 0.9893947002666886
CV score for class insult is 0.9823207285800888
CV score for class identity_hate is 0.9824677282031388
Total CV score is 0.9850812550400473


In [56]:
submission.to_csv('submission_lr.csv', index=False)